# Insurance premium calculator

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import calendar
import datetime
from datetime import date
import lmoments
import statistics
from scipy import stats

In [1]:
#Read data
df=pd.read_csv('/syntheticseries.csv',sep=',')
df['Datetime']=pd.to_datetime(df['timestamp.1'])
df=df.set_index(['Datetime'])
df.head()

In [4]:
#Calculate the indice
#in this case: Two year of accumulated precipitation data
df2 = df.rolling('730d').sum()
df2.to_csv(r'P24-daily.csv', sep=',')
df2.head()


In [6]:
#Define the threshold and calculate the number of exceedances
#in this case the threshold is: 2424 mm
#if the value is bellow the threshold (for droughts), the loss is R$ 775229.65 (which corresponds to the calculated value in R$ of the damage caused by the drought per day) 
def filter (df2,columnname):
    df2.loc[df2[columnname]<=2424,columnname+'_'+'diff']=775229.65 
    df2.loc[df2[columnname]>2424,columnname+'_'+'diff']=0


In [7]:
#Apply the filter in all the columns
columnname = df2.columns
for c in columnname:
    filter(df2,c)

In [8]:
#Save only the columns with the suffix _diff
##Salva apenas as colunas novas com o sufixo 'diff'
drycolumns=df2.filter(regex='diff')

#Select the period of analysis
dry=drycolumns.loc['1985-01-01':'2013-12-31'] 

In [9]:
#Payoff calculation - Annualy accumulated losses.
payoffannual=dry.resample('Y').sum()
payoffannual.to_csv(r'payoff.csv', sep=',')

In [10]:
#Define the maximum payout function
#This is arbitrary
Max=282958820.64
def maxpayoff(payoff,columnname):
    payoff.loc[payoff[columnname]>=Max,columnname]=Max
    payoff.loc[payoff[columnname]<Max,columnname]

In [ ]:
#Apply the maximum payout function for all columns and save the results
columnname = payoffannual.columns
for c in columnname:
    maxpay=maxpayoff(payoffannual,c)
payoffannual.to_csv(r'payofftotal', sep=',', columns=maxpay)